<a href="https://colab.research.google.com/github/HSS107048212/Neil/blob/main/Neil_GATcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import copy
import math
import os

In [ ]:
patent_data = pd.read_excel('c9-2.xlsx', header=1) #這一個要放蒐集下來的核心專利
patent_data_family = pd.read_excel('c9-family.xlsx', header=1) #這一個要放核心專利的專利家族和集
patent_data = patent_data.fillna('')

#下面是將專利家族的引用關係複製到核心專利那個上面，由於在建構引用網路時重複的關係並不會被計算到，因此採直接複製
#下面這段是在複製DWPI family的資料

for i in patent_data.index: #整理 Cited Refs - Patent 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = patent_data_family['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index: # 整理 Citing Patents 欄位
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in patent_data_family['Publication Number'].values:
                patent_index = patent_data_family.index[patent_data_family['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = patent_data_family['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c


#下面是做一樣的事情，只是變成複製INPADC的資料，單純因為懶，所以分成兩個比較方便
INPADC1 = pd.read_excel('INPADC1.xlsx', header=1)
INPADC2 = pd.read_excel('INPADC2.xlsx', header=1)
INPADC3 = pd.read_excel('INPADC3.xlsx', header=1)
INPADC4 = pd.read_excel('INPADC4.xlsx', header=1)
INPADC = pd.concat([INPADC1,INPADC2])
INPADC = pd.concat([INPADC,INPADC3])
INPADC = pd.concat([INPADC,INPADC4])
INPADC.index = list(range(len(INPADC)))

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'DWPI Family Members']):
        temp = patent_data.loc[i,'DWPI Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Cited Refs - Patent'][i]
                    b = INPADC['Cited Refs - Patent'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Cited Refs - Patent'][i] = c

for i in patent_data.index:
    if pd.notnull(patent_data.loc[i,'INPADOC Family Members']):
        temp = patent_data.loc[i,'INPADOC Family Members'].split('|')
        c = ''
        for j in temp:
            j=j.strip()
            if j in INPADC['Publication Number'].values:
                patent_index = INPADC.index[INPADC['Publication Number']==j].tolist()
                for k in patent_index:
                    k = int(k)
                    a = patent_data['Citing Patents'][i]
                    b = INPADC['Citing Patents'][k]
                    if pd.isna(a) == False and pd.isna(b) == False:
                        c = a + '|' + b
                        patent_data['Citing Patents'][i] = c

#最後將引用關係合併之後的資料集輸出為excel，在後續分析時要用
patent_data.to_excel('c9-append.xlsx')

<ipython-input-4-b012f9d53a5c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Cited Refs - Patent'][i] = c
<ipython-input-4-b012f9d53a5c>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patent_data['Citing Patents'][i] = c


In [ ]:
patent_data

,Publication Number,Title (English),Assignee/Applicant,Publication Year,Publication Date,Abstract (English),Abstract - DWPI,Cited Refs - Patent,Citing Patents,Count of Citing Patents,Count of Cited Refs - Patent,IPC Current Full (4 Characters),INPADOC Family Members,DWPI Family Members
0,TW201818743A,ROUTING METHOD AND WIRELESS NODE FOR WIRELESS ...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2018,2018-05-16,A routing method for a wireless mesh network a...,The method involves performing (S610) route al...,US7159035B2 | US8488589B2 | US8948046B2 | US90...,CN113259940A | CN113259940B | TWI727786B|CN114...,3,4,"H04L, H04W",TW201818743A | TWI604744B | US10250503B2 | US...,US20180123957A1 | TWI604744B | TW201818743A |...
1,TW201946421A,TRANSPARENT BRIDGING OVER MESH NETWORK,"GUIZHOU DINGTONGXIN TECHNOLOGIES LTD.,CN | 大陸商...",2019,2019-12-01,A method of transparent bridging over mesh net...,The method involves receiving a packet from an...,US20170034760A1 | US8730809B2 | US20130100872A...,,0,4,H04L,TW201946421A | CN110417659A | TWI731237B,CN110417659A | TW201946421A | TWI731237B
2,TW201715902A,MANAGEMENT METHOD FOR WIRELESS NETWORK AND NET...,"INDUSTRIAL TECHNOLOGY RESEARCH INSTITUTE,TW | ...",2017,2017-05-01,A wireless network management method for a wir...,A wireless network management method for a wir...,US20110255570A1 | US20140153551A1 | US20140119...,WO2019127204A1|WO2019127204A1|US10356759B2 | U...,1,3,H04W,TW201715902A | TWI568279B | US20170118752A1 |...,TW201715902A | TWI568279B | US9999045B2 | US2...
3,TW202245502A,METHOD AND SYSTEM FOR UPDATING DEVICE FIRMWARE...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A firmware update method. On a wireless mesh n...,A firmware update method. On a wireless mesh n...,CN106713047A | CN102158851B | CN103685530A|CN1...,,0,3,"H04W, H04L, G06F",,TW202245502A | TWI784506B
4,TW202245501A,METHOD AND SYSTEM FOR REMOTELY CONTROLLING FIR...,"COMPAL BROADBAND NETWORKS INC.,TW | 鋐寶科技股份有限公司,TW",2022,2022-11-16,A method or system for remotely controlling th...,A method or system for remotely controlling th...,CN103685530A | CN106713047A | CN102158851B|CN1...,,0,3,"H04W, H04L, G06F",,TW202245501A | TWI784505B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,TW202114382A,ROUTER AND ROUTING METHOD,"REALTEK SEMICONDUCTOR CORPORATION,TW | 瑞昱半導體股份...",2021,2021-04-01,The application discloses a router coupled to ...,The router (100) has a switch module (112) pro...,US8565240B2 | US20030016628A1 | WO2013081511A1...,,0,4,H04L,TW202114382A | TWI730476B | US11206568B2 | US2...,US20210092633A1 | TW202114382A | TWI730476B | ...
92,TW202113749A,SYSTEM FOR INTEGRATED MANAGEMENT OF VIRTUAL AN...,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-04-01,The present invention is a system for integrat...,The present invention is a system for integrat...,TW201527991A | TW201911045A | TW200611112A | T...,,0,5,"G06Q, H04L",TW202113749A | TWI709941B,TWI709941B | TW202113749A
93,TW202121869A,MANAGEMENT SYSTEM AND MANAGEMENT METHOD OF VNF,"CHUNGHWA TELECOM CO. LTD.,TW | 中華電信股份有限公司,TW",2021,2021-06-01,A management system and management method of V...,A management system and management method of V...,TW201909601A | WO2015106822A1 | US20190058636A...,|TWI789014B | TWI795123B,0,7,H04L,TW202121869A | TWI707561B,TWI707561B | TW202121869A
94,TW202121917A,CHANNEL LOADING PRE-ADJUSTING SYSTEM FOR 5G WI...,"ALPHA NETWORKS INC.,TW | 明泰科技股份有限公司,TW",2021,2021-06-01,A channel loading pre-adjusting system for 5G ...,The channel loading pre-adjusting system (10) ...,TW201212693A | US20170332270A1 | US20080134194...,|CN113965946A,0,3,H04W,TW202121917A | CN112839357A | CN112839357B | T...,US20210160734A1 | CN112839357A | TWI701956B | ...


In [ ]:
import pandas as pd
import numpy as np
import copy

import os

#從Derwent下載下來的資料，核心專利、核心專利引用的專利跟引用核心專利的專利
#patent_data = pd.read_excel('1215_append.xlsx', header=0) #這個是要使用合併引用關係之後的核心資料集
#cited_data = pd.read_excel('1215_cited.xlsx', header=1)
#citing_data = pd.read_excel('1215_citing.xlsx', header=1)

# Neil學長的 已经将Excel 文件转换为了 CSV 格式，并且文件名保持一致，只是扩展名变为了 .csv
patent_data = pd.read_csv('1215_append.csv', header=0, encoding='utf-8') # 这个是要使用合并引用关系之后的核心数据集
cited_data = pd.read_csv('1215_cited.csv', header=2, encoding='utf-8')
citing_data = pd.read_csv('1215_citing.csv', header=2, encoding='utf-8')

#cited_data = cited_data.iloc[3:]
#citing_data = citing_data.iloc[3:]
print(patent_data.columns)
print(cited_data.columns)
print(citing_data.columns)


#整理資料，保留特定欄位並合併三個資料集為全部使用到的大資料集。
temp_patent = patent_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents'] ]
temp_cited = cited_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
temp_citing = citing_data[['Publication Number','Title (English)','Assignee/Applicant','Publication Year','Publication Date','Abstract (English)','Abstract - DWPI','Cited Refs - Patent','Citing Patents','Count of Cited Refs - Patent','Count of Citing Patents']]
all_patent = pd.concat([temp_patent,temp_cited])
all_patent = pd.concat([all_patent,temp_citing])
all_patent = all_patent.dropna(subset=['Title (English)','Abstract (English)','Abstract - DWPI']) #將三行都是空值的列刪除
#刪除重複值並用Publication Year排序
all_patent = all_patent.drop_duplicates()

# 将 'Publication Year' 列转换为整数，无法转换的设置为 NaN
all_patent['Publication Year'] = pd.to_numeric(all_patent['Publication Year'], errors='coerce')

# 现在可以排序了，因为所有数据都是整数类型
all_patent = all_patent.sort_values('Publication Year')

# 如果你想要删除 'Publication Year' 列中的 NaN 值
all_patent = all_patent.dropna(subset=['Publication Year'])

all_patent = all_patent.sort_values('Publication Year')
all_patent1 = all_patent.drop_duplicates(subset=['Publication Number'])
all_patent1.index = list(range(len(all_patent1)))



Index(['Unnamed: 0', 'Publication Number', 'Title (English)',
       'Assignee/Applicant', 'Publication Year', 'Publication Date',
       'Abstract (English)', 'Abstract - DWPI', 'Cited Refs - Patent',
       'Citing Patents', 'Count of Citing Patents',
       'Count of Cited Refs - Patent', 'IPC Current Full (4 Characters)',
       'INPADOC Family Members', 'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Patents',
       'Count of Citing Patents', 'Count of Cited Refs - Patent',
       'IPC Current Full (4 Characters)', 'INPADOC Family Members',
       'DWPI Family Members'],
      dtype='object')
Index(['Publication Number', 'Title (English)', 'Assignee/Applicant',
       'Publication Year', 'Publication Date', 'Abstract (English)',
       'Abstract - DWPI', 'Cited Refs - Patent', 'Citing Pate

<ipython-input-1-179338ce6ccf>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_patent['Publication Year'] = pd.to_numeric(all_patent['Publication Year'], errors='coerce')


In [7]:
import pandas as pd
import pandas as pd
file_path = '/content/Mickey_PCB_world _core.xlsx'
all_patent1 = pd.read_excel(file_path)


In [8]:
import numpy as np
#合併會使用到的專利文字資料，並在原資料及創造一個新的欄位名叫text存放
text = []
for i in all_patent1.index:

  str1 = str(all_patent1.loc[i,'Title (English)'])+' '+str(all_patent1.loc[i,'Abstract (English)'])+' '+str(all_patent1.loc[i,'Abstract - DWPI'])

  str2 = ''.join(str1.split('nan'))
  text.append(str2)
text = np.array(text)
all_patent1['text'] = text

#有text的專利資料集先存起來備用
all_patent1.to_excel('Mickey_PCB_world _core_text.xlsx')

20231227test

In [4]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=21f9ab1bb03b4832e640b216d20322e17d0f63795fb90bdedbe9e2ca9fb87663
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
pip install corextopic

In [12]:
import pandas as pd
import numpy as np  # 需要導入 NumPy 库
# 指定 Excel 檔案的路徑
excel_file_path = 'Mickey_PCB_world _core_text.xlsx'

# 讀取 Excel 檔案
df = pd.read_excel(excel_file_path)
text = np.array(df['text'])

In [13]:
#Sentence Transformer 將文字轉成向量，paraphrase-distilroberta-base-v1為在英文上表現最好BERT的模型
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
text_embeddings = model.encode(text)
text_embeddings_df = pd.DataFrame(text_embeddings)
#將文字向量存起來，之後就不需要再跑第二次
text_embeddings_df.to_csv('text_embeddings_title_abstract_new.csv',index=False)

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
#-------------------------------------------------------------------------------接下來是進行主題模型
import  scipy.sparse as ss
import matplotlib.pyplot as plt
import corextopic.corextopic as ct
import corextopic.vis_topic as vt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#讀取存有專利文字資料的資料集
patents = pd.read_excel('IoV new_M.xlsx')

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 新增區段

In [ ]:
text = patents['text'].values
#先刪除一些不在常規停用字的停用字，這邊可以自由新增減少，從關鍵字的結果中可以判斷，通常專利中有很多垃圾字，需要先處理過後再跑主題模型會好很多。
stopwords = ['aspects of subject','non-English','non english','flow chart','block diagram','copyright', 'jpo', 'japio','100wireless','Drawing','includes','language','text','drawing','claims','claim','following','english','aa','shows','figure','method','methods','flowchart',
             'illustrating','involves','diagram','schematic','block','chart','explanatory view','representation','determined','determining','realize','realizing','invention','according','technical','comprises','field','realizes','problem','problems','solves','module','solving','solves','embodiment',
             'described','associated','systems','display','allows','providing','including','kipo','copyright']
for i in range(len(text)):
    text[i] = text[i].lower()
    for j in stopwords:
        if j in text[i]:
            temp_text = text[i].split(j)
            text[i] = "".join(temp_text)

#使用CountVectorizer，先將常規的停用字刪除，然後將文字資料轉換成1-gram與2-gram的詞，在這些詞中由出現頻率作為依據取前100000個詞
vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2),max_features=100000,binary=True)
doc_word = vectorizer.fit_transform(text)
doc_word = ss.csr_matrix(doc_word)
# 提取出現頻率100000個1-gram與2-gram的詞
words = list(np.asarray(vectorizer.get_feature_names()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words = [word for ind,word in enumerate(words) if not word.isdigit()]

#使用CorEX模型提取主題，n_hidden為主題數，words生成主題使用到的詞，max_iter為CorEX迭代的次數
topic_num = 25  #生成的主體數，需要自己去調控
topic_model = ct.Corex(n_hidden=topic_num,words = words,max_iter = 200,verbose=False,seed = 2) #這邊是corex的參數，可調可不調，看結果
topic_model.fit(doc_word,words=words)
#每個主題提取前50個字
topics = topic_model.get_topics(n_words=50)
keywords = []
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    temp_str = '{}: '.format(n) + ', '.join(topic_words)
    keywords.append(temp_str)
keywords_prob = []
for i in range(topic_num):
    temp = []
    for j in topic_model.get_topics(topic=i, n_words=50):
        temp.append(str(j))
    keywords_prob.append(temp)
#輸出CorEX主題相關資訊
plt.figure(figsize=(10,5))
plt.bar(range(topic_model.tcs.shape[0]), topic_model.tcs, color='#4e79a7', width=0.5)
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16);
plt.savefig("mygraph_24_anchor.png")

In [ ]:
#下面的prefix代表了輸出主題模型資訊的資料夾名稱，最好可以改成自己認識的字
vt.vis_rep(topic_model, column_label=words, prefix='new IoV patents 30topics 1')
many_keywords_prob = []
for i in range(topic_num):
    temp = []
    for j in topic_model.get_topics(topic=i, n_words=100):
        temp.append(str(j))
    many_keywords_prob.append(temp)
with open('100topic_keyword_prob_patents.txt','w',encoding='utf8') as f:
   for i in many_keywords_prob:
       f.write(str(i))
       f.write('\n')
   f.close()
topics = topic_model.get_topics(n_words=100)
many_keywords = []
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    temp_str = '{}: '.format(n) + ', '.join(topic_words)
    many_keywords.append(temp_str)

with open('100topic_keyword_patents.txt','w',encoding='utf8') as f:
    for i in many_keywords:
        f.write(i)
        f.write('\n')
    f.close()


In [ ]:
with open('labels.txt','r') as f: #這邊的labels，在輸出主題模型資訊的資料夾中可以找到，代表的是專利與其代表主題的號碼
  label = f.read()
  label = label.split('\n')
  f.close()
topic_labels = []
for i in label[:-1]:
  temp = i.split(',')
  topic_labels.append([all_patent1.loc[int(temp[0]),'Publication Number'],temp[1:]])
topic_labels = np.array(topic_labels)
topic_value = []
for i in np.array(topic_labels[:,1]):
  temp = []
  for j in i:
    temp.append(int(j))
  topic_value.append(temp)
topic_value = np.array(topic_value)
topic_value

In [9]:
pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=0685578e739c4dbd4c03e71f370da137062d6bebb50c2bb7ae6de61c0e3b0e25
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
Successfully built keybert


In [10]:
#-------------------------------------------------------------------------------以下是進行keybert
import re
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
#對text進行前處理
text = all_patent1['text'].values
text_process = []
for t in text:
    remove_chars = '[0-9’"#$%&\'()*+-./:;<=>?@。?★、…【】《》？“”‘’[\\]^_`{|}~]+'
    t = re.sub(remove_chars, '', t)
    final_text = ''
    for i in t.split():
        if 'jpo' and 'japio' not in i:
            final_text += i
            final_text += ' '

    text_process.append(final_text)
#導入KeyBERT模型
#使用KeyBERT模型時，因為需要提取關鍵字的資料量很大，因此設置GPU加速運算
#sentence_model = SentenceTransformer("all-MiniLM-L6-v2", device='cuda')
model = KeyBERT("all-MiniLM-L6-v2")
import torch
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(dev)

keywords = []

for i in range(len(text_process)):
  #每篇文檔關鍵詞可設定n-gram、前幾個關鍵詞、關鍵詞的多樣性
  keyword = model.extract_keywords(text_process[i],keyphrase_ngram_range=(1, 2), top_n=10,use_mmr=True, diversity=0.6)
  print(i)
  keyword = np.array(keyword)
  if keyword.size != 0:
    keywords.append(keyword[:,0])
    temp_str = ','.join(keyword[:,0])
  else:
    keywords.append(keyword)
    temp_str = ''
  with open('keywords_patents_new_IoV_diversity_0.6.txt','a',encoding='utf8') as f:

    f.write(temp_str)
    f.write('\n')
f.close()

#all_patent1['keywords'] = np.array(keywords)
all_patent1['keywords'] = keywords
all_patent1.to_excel('all_patent1_keywords.xlsx')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

串流輸出內容已截斷至最後 5000 行。
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225

In [14]:
#all_patent1 = pd.read_excel('all_patent1_keywords.xlsx')
#all_patent1 = np.array(all_patent1_keywords['keywords'])
#------------------------------------------------------------------------------以下要開始進行訓練模型的準備
#定義function使專利資料分成訓練集、測試集與驗證集，使用mask遮蓋資料。mask由(True,False)組成，True為此資料被分到訓練集/測試集/驗證集，False代表沒有，訓練集/測試集/驗證集的數量跟專利數量一樣
def split_train_test_val(data,test_ratio,val_ratio):
  np.random.seed(52)
  shuffled_indices = np.random.permutation(len(data))
  test_size = int(len(data)*test_ratio)
  val_size = int(len(data)*val_ratio)
  test_incides = shuffled_indices[:test_size]
  val_incides = shuffled_indices[test_size:test_size+val_size]
  train_incides = shuffled_indices[test_size+val_size:]
  test_mask = []
  for i in range(len(data)):
    if i in test_incides:
      test_mask.append(True)
    else:
      test_mask.append(False)
  val_mask = []
  for i in range(len(data)):
    if i in val_incides:
      val_mask.append(True)
    else:
      val_mask.append(False)
  train_mask = []
  for i in range(len(data)):
    if i in train_incides:
      train_mask.append(True)
    else:
      train_mask.append(False)
  return np.array(train_mask), np.array(test_mask),np.array(val_mask)
train_mask,test_mask,val_mask = split_train_test_val(all_patent1['Publication Year'].values,0.2,0.1)

#將Publication Year標準化，並將標準化的結果跟每篇文字向量合併當成專利的特徵向量
Publication_Year = all_patent1['Publication Year'].values
Publication_Year[np.isnan(Publication_Year)] = 0
Publication_Year = np.reshape(Publication_Year,(len(all_patent1),1))
Publication_Year.shape
Publication_Year_normalization = (Publication_Year - np.min(Publication_Year))/(np.max(Publication_Year)-np.min(Publication_Year))
Publication_Year_normalization
node_feature = np.append(Publication_Year_normalization,text_embeddings,axis=1)
# 將專利發行人利用發行人的頻率進行轉換並將每篇專利的發行人編碼，然後將每篇專利的發行人編碼合併進專利的特徵向量
assignee = []
for i in all_patent1['Assignee/Applicant'].values:
  if type(i) != str:
    temp = str(i)
    assignee.append(temp)
  else:
    assignee.append(i)
assignee = np.array(assignee)
all_patent1['Assignee/Applicant'] = assignee
enc_nom_1 = (all_patent1.groupby('Assignee/Applicant').size()) / len(all_patent1)
enc_nom_1['nan'] = 0
all_patent1['Assignee/Applicant_encode'] = all_patent1['Assignee/Applicant'].apply(lambda x : enc_nom_1[x])
Assignee_Applicant_encode = all_patent1['Assignee/Applicant_encode'].values
Assignee_Applicant_encode = np.reshape(Assignee_Applicant_encode,(len(all_patent1),1))
node_feature = np.append(Assignee_Applicant_encode,node_feature,axis=1)
node_feature.tofile('node_feature.csv', sep = ',')

In [16]:
# 安裝PyG套件，可先檢查環境配置以安裝相對應的PyG版本
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 939.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=915fff340b0c1ea1398bebe355499ee4aa90f9212d3de316e9f7ee724d1bc8fd
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter


In [17]:
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1035675 sha256=bbb890c3c47e10e0e9607e6193a223a3822b3e471d76086e00c7993ba216e4e3
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built torch-sparse


In [18]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00


In [26]:
#networkx套件專門處理graph
#下面是在構築引用網路
import networkx as nx

all_patent1.index = range(len(all_patent1))
#先宣告一個具有方向性的graph架構，將專利的Publication Number當成節點放入，並將專利的特徵向量，訓練/測試/驗證資料集的mask，還有每篇專利的主題一起當成graph的特徵放入
G = nx.DiGraph()

#G.add_node(0, feature=5, label=0)
print(G.is_directed())
for i in list(all_patent1.index):
  G.add_node(
    all_patent1.loc[i,'Publication Number'],
    publication_number = all_patent1.loc[i,'Publication Number'],
    x = node_feature[i],
    train_mask = train_mask[i],
    test_mask = test_mask[i],
    val_mask = val_mask[i],
    y = 8
    # y = topic_value[i]
    )

#有了節點後，再將節點間的邊放入，用兩個角度來看，一個是從所有專利的Citing Patents欄位，如果Citing Patents中的專利有在我們的專利資料中，則在節點專利與其Citing Patents裡的專利節點中間增加一條邊，方向為節點專利指向其Citing Patents裡的專利節點
for i in list(all_patent1.index):
  if pd.notnull(all_patent1.loc[i,'Citing Patents']):
    temp = all_patent1.loc[i,'Citing Patents'].split('|')
    for j in temp:
      j = j.strip()
      if j in all_patent1['Publication Number'].values:
        G.add_edge(all_patent1.loc[i,'Publication Number'],j)
#再來是從專利的Cited Refs - Patent欄位，如果Cited Refs - Patent中的專利有在我們的專利資料中，則在Cited Refs - Patent裡的專利節點與其所對應的專利節點增加一條邊，方向為Cited Refs - Patent裡的專利節點指向其所對應的專利節點
for i in list(all_patent1.index):
  if pd.notnull(all_patent1.loc[i,'Cited Refs - Patent']):
    temp = all_patent1.loc[i,'Cited Refs - Patent'].split('|')
    for j in temp:
      j = j.strip()
      if j in all_patent1['Publication Number'].values:
        G.add_edge(j,all_patent1.loc[i,'Publication Number'])

#可用來檢查圖的內容
print("Total number of nodes: ", int(G.number_of_nodes()))
print("Total number of edges: ", int(G.number_of_edges()))
print("List of all nodes: ", list(G.nodes()))
print("List of all edges: ", list(G.edges()))
print("In-degree for all nodes: ", dict(G.in_degree()))
print("Out degree for all nodes: ", dict(G.out_degree))

#下面的三個套件是要使用pytorch來訓練神經網路時的必要套件，需要是先按裝，友情提醒，pytorch cuda版本與下面三個套件的版本對照非常重要


import torch
from torch_geometric.data import Data
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
#將networkx型態的圖資料轉成訓練以pytorch為基礎的神經網路所需資料型態
data2 = from_networkx(G)
#看data2基本資訊
print(data2)
print(f'Number of nodes: {data2.num_nodes}')
print(f'Number of edges: {data2.num_edges}')
print(f'Average node degree: {(2*data2.num_edges) / data2.num_nodes:.2f}')
#print(f'Number of training nodes: {data2.train_mask.sum()}')
#print(f'Training node label rate: {int(data2.train_mask.sum()) / data2.num_nodes:.2f}')
print(f'Contains isolated nodes: {data2.has_isolated_nodes()}')
print(f'Contains self-loops: {data2.has_self_loops()}')
print(f'Is undirected: {data2.is_undirected()}')

#導入PyG等套件，並建立gat的主要架構，這邊如要換模型又不知道怎麼用，可以去搜尋pyg的github上面有範例
from torch.nn import Linear, ELU
from torch_geometric.nn import GATConv
from torch import Tensor

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels , heads, dropout = 0.3)
        self.drop = torch.nn.Dropout(0.3)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, head = 1, concat=False, dropout=0.3)


    def forward(self, x, edge_index) :
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        h = self.conv1(x, edge_index)
        h = self.drop(h).relu()
        h = self.conv2(h,edge_index).sigmoid()# Final GNN embedding space.
        return h


"""
model = torch.load('model_GAT_85.pth')
model.eval()
torch.no_grad()
h = model((data2.x).float(), data2.edge_index)
h = h.float()
predicted_classes = (h>0.5)*1
b = predicted_classes[72147:]  #這邊由於預測時需要將所有節點一起放入，因此只取需要的後半段
b = pd.DataFrame(b.numpy())
b.to_excel('classes_c10_original.xlsx') #這邊的b可以將預設結果輸出為excel
"""

True
Total number of nodes:  7864
Total number of edges:  71266
List of all nodes:  ['CN111032917B', 'US10064284B2', 'CN107849717A', 'CN105307380B', 'EP4145964A1', 'CN115135806B', 'US9526178B2', 'JP2015019568A', 'KR2466583B1', 'CN108326801B', 'EP4056000A1', 'KR2022029557A', 'US11225722B2', 'US11647590B2', 'US11452214B2', 'US11399432B2', 'TW202339363A', 'US10087368B2', 'WO2021174886A1', 'EP3284325B1', 'US9827616B2', 'CN103945636B', 'US20210315104A1', 'CN107881468B', 'JP2015039002A', 'US10763002B2', 'KR2023156815A', 'CN108617099B', 'KR2023129046A', 'EP3507853B1', 'TWI800804B', 'JP2023073993A', 'KR2019132959A', 'KR2587271B1', 'TWM569541U', 'EP3772239B1', 'CN108419432B', 'TWI630261B', 'CN112629294B', 'CN211700630U', 'US9480172B2', 'JP2023116478A', 'TW202319839A', 'TW202130243A', 'MX339806B', 'US20220225500A1', 'KR2023144653A', 'EP4245886A1', 'EP4007463A4', 'US10933490B2', 'CN105281063B', 'CN117043440A', 'KR2365234B1', 'CN110844931B', 'KR1917018B1', 'US9258885B2', 'CN105887144B', 'CN1057049

"\nmodel = torch.load('model_GAT_85.pth')\nmodel.eval()\ntorch.no_grad()\nh = model((data2.x).float(), data2.edge_index)\nh = h.float()\npredicted_classes = (h>0.5)*1\nb = predicted_classes[72147:]  #這邊由於預測時需要將所有節點一起放入，因此只取需要的後半段\nb = pd.DataFrame(b.numpy())\nb.to_excel('classes_c10_original.xlsx') #這邊的b可以將預設結果輸出為excel\n"

In [32]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT(data2.num_features, 256,(data2.y).shape[-1],7)
criterion = torch.nn.BCELoss()  # 定義loss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)  # 定義 optimizer.

#定義訓練function
def train(data2):
  best_accuracy = 0.0
  optimizer.zero_grad()  # Clear gradients.

  h = model((data2.x).float(), data2.edge_index)  # Perform a single forward pass.
  h = h.float()

  loss = criterion((h[data2.train_mask]).float(), (data2.y[data2.train_mask]).float())  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.

  accuracy = {}
  # Calculate training accuracy on training data2set
  predicted_classes = (h[data2.train_mask]>0.5)*1 # [0.6, 0.2, 0.7, 0.1] -> 2

  #torch.argmax(out[data2.train_mask], axis=1)
  target_classes = data2.y[data2.train_mask]
  accuracy['train'] = torch.mean(
      torch.where(predicted_classes == target_classes, 1, 0).float())

  # Calculate validation accuracy on the validation set
  predicted_classes = (h[data2.val_mask]>0.5)*1
  with torch.no_grad():
    model.eval()
    val_target_classes = data2.y[data2.val_mask]
    accuracy['val'] = torch.mean(
      torch.where(predicted_classes == val_target_classes, 1, 0).float())

  # Calculate whole accuracy on the whole graph
  predicted_classes = (h>0.5)*1
  target_classes = data2.y
  accuracy['all accuracy'] = torch.mean(
      torch.where(predicted_classes == target_classes, 1, 0).float())
  return loss,h, accuracy

for epoch in range(100):
    loss, h, accuracy = train(data2)
    print('loss:', loss, ' ', 'accuracy: ',accuracy)

#將訓練好的模型參數存下來，之後預測要用
torch.save(model, 'model_GAT_85.pth')


RuntimeError: ignored

In [28]:
data2.num_features

770